In [1]:
!pip3 install scikit-optimize
%matplotlib inline

import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# distributions
from scipy.stats import randint 
from scipy.stats import uniform

#Bayessearch cv uit Scikit-optimize
from skopt import BayesSearchCV


from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
# Import Keras libraries

# Import Tensorflow libraries
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import dataset
dataset = pd.read_csv('cancer.csv')
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
# Data preprocessing
dataset.drop('id',axis=1, inplace=True)

# Remove right column (unnamed)
dataset.drop(dataset.columns[31],axis=1,inplace=True)

# diagnosis labels M -> 1 and B -> 0
dataset.diagnosis.replace(['M', 'B'], [1, 0], inplace=True)

/var/folders/0r/s3s7wy7x0z33qbtj01kxq24w0000gn/T/ipykernel_63094/867236065.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset.diagnosis.replace(['M', 'B'], [1, 0], inplace=True)
/var/folders/0r/s3s7wy7x0z33qbtj01kxq24w0000gn/T/ipykernel_63094/867236065.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasti

In [4]:
# Split dataset up in features and targets
y = dataset.diagnosis
X = dataset.drop('diagnosis',axis=1)

In [5]:
# Create training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state =0)

# feature normalization
# via robustscaler: This Scaler removes the median and scales the data according to the quantile range 

Rscaler = RobustScaler()
Rscaler.fit(X_train)

X_train = Rscaler.transform(X_train)
X_test = Rscaler. transform(X_test)


In [6]:
# Train an SVM (grid search)

model = SVC()
parameters = [ 
        {'kernel': ['linear'], 'C': np.linspace(1,20,10)},
        {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2]},
        {'kernel': ['poly'], 'C':[1, 10]} ]
grid_search = GridSearchCV(estimator = model, 
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4,
                           n_jobs = -1,
                           verbose = 3)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  

print('Best accuracy : ', grid_search.best_score_)
print('Best parameters :', grid_search.best_params_  )

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 




Fitting 4 folds for each of 22 candidates, totalling 88 fits
[CV 1/4] END ..............C=1.0, kernel=linear;, score=0.991 total time=   0.0s
[CV 2/4] END ..............C=1.0, kernel=linear;, score=0.963 total time=   0.0s
[CV 3/4] END ..............C=1.0, kernel=linear;, score=0.991 total time=   0.0s
[CV 4/4] END ..............C=1.0, kernel=linear;, score=1.000 total time=   0.0s
[CV 1/4] END C=7.333333333333334, kernel=linear;, score=0.981 total time=   0.0s
[CV 2/4] END C=7.333333333333334, kernel=linear;, score=0.935 total time=   0.0s
[CV 3/4] END C=7.333333333333334, kernel=linear;, score=0.953 total time=   0.0s
[CV 4/4] END C=7.333333333333334, kernel=linear;, score=0.981 total time=   0.0s
[CV 1/4] END C=9.444444444444445, kernel=linear;, score=0.981 total time=   0.0s
[CV 2/4] END C=9.444444444444445, kernel=linear;, score=0.935 total time=   0.0s
[CV 4/4] END C=3.111111111111111, kernel=linear;, score=0.991 total time=   0.0s
[CV 3/4] END C=9.444444444444445, kernel=linear;

In [7]:
# Train an SVM (random search)

model = SVC()
parameters = {'kernel': ['linear','rbf','poly'],
              'C': uniform(0.01, 100), 
              'gamma': uniform(0.001, 2)}


n_iter_search = 40


random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=4,n_iter=n_iter_search)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  

print('Best accuracy : ', random_search.best_score_)
print('Best parameters :',random_search.best_params_  )

y_pred = random_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 




Best accuracy :  0.9624625286545583
Best parameters : {'C': 12.10473875457283, 'gamma': 0.2848947169664814, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        90
           1       0.96      0.92      0.94        53

    accuracy                           0.96       143
   macro avg       0.96      0.95      0.95       143
weighted avg       0.96      0.96      0.96       143

[[88  2]
 [ 4 49]]
95.8041958041958


In [8]:
# Train an SVM (Bayes optimization )

model = SVC()
parameters = {'kernel': ['linear','rbf','poly'],
              'C': (0.01, 100,'uniform'), # 
              'gamma': (0.001, 2,'uniform')}

n_iter_search = 20

Bayes_search = BayesSearchCV(model,parameters,n_iter=n_iter_search,cv=4,verbose=1)


Bayes_search.fit(X_train, y_train)

best_accuracy = Bayes_search.best_score_ 
best_parameters = Bayes_search.best_params_  

print('Best accuracy : ', Bayes_search.best_score_)
print('Best parameters :',Bayes_search.best_params_  )

y_pred = Bayes_search.predict(X_test)
print(classification_report(y_test, y_pred))

cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 



Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fi

# Neural Network hyperparametertuning

In [9]:
# Import dataset
dataset = pd.read_csv('cancer.csv')
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [10]:
# Data preprocessing
dataset.drop('id',axis=1, inplace=True)

# Remove right column (unnamed)
dataset.drop(dataset.columns[31],axis=1,inplace=True)

# diagnosis labels M -> 1 and B -> 0
dataset.diagnosis.replace(['M', 'B'], [1, 0], inplace=True)


# Split dataset up in features and targets
y = dataset.diagnosis
X = dataset.drop('diagnosis',axis=1)



/var/folders/0r/s3s7wy7x0z33qbtj01kxq24w0000gn/T/ipykernel_63094/4164977983.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset.diagnosis.replace(['M', 'B'], [1, 0], inplace=True)
/var/folders/0r/s3s7wy7x0z33qbtj01kxq24w0000gn/T/ipykernel_63094/4164977983.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcas

In [11]:
# Create training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state =0)

# Convert to numpy array
X_train = X_train.values
X_test = X_test.values

# One-hot encoding of the targets
y_train = to_categorical(y_train)

# Feature normalization
# via robustscaler: This Scaler removes the median and scales the data according to the quantile range 

Rscaler = RobustScaler()
Rscaler.fit(X_train)

X_train = Rscaler.transform(X_train)
X_test = Rscaler. transform(X_test)

In [12]:
def create_model(optimizer='adam',activation = 'relu',dropout_rate = 0.0, kernel_initializer='uniform',neurons = 10):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], kernel_initializer=kernel_initializer,activation="relu"))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(neurons, kernel_initializer=kernel_initializer,activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(neurons, kernel_initializer=kernel_initializer,activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=['accuracy']) 
    return model

In [13]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define the model
model = KerasClassifier(build_fn=create_model, batch_size=32, epochs=10)

# Define the parameter grid
param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 20],
    'optimizer': ['Adam', 'SGD']
}

# Create the GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=3)

# Retrieve the parameter keys
print(grid.get_params().keys())

dict_keys(['cv', 'error_score', 'estimator__model', 'estimator__build_fn', 'estimator__warm_start', 'estimator__random_state', 'estimator__optimizer', 'estimator__loss', 'estimator__metrics', 'estimator__batch_size', 'estimator__validation_batch_size', 'estimator__verbose', 'estimator__callbacks', 'estimator__validation_split', 'estimator__shuffle', 'estimator__run_eagerly', 'estimator__epochs', 'estimator__class_weight', 'estimator', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])


In [21]:
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(optimizer="adam", activation="relu"):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=activation, input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(2, activation="softmax")  # 2 classes, one-hot targets
    ])
    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model

clf = KerasClassifier(model=create_model, verbose=0)

param_grid = {
    "batch_size": [8, 16],
    "epochs": [10],
    "model__optimizer": ["SGD", "adam"],
    "model__activation": ["relu", "tanh"],
}

grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, verbose=3, error_score="raise")
grid_result = grid.fit(X_train, y_train)  # y_train is one-hot (n,2)

print("Best Parameters:", grid_result.best_params_)
print("Best Score:", grid_result.best_score_)


Fitting 3 folds for each of 8 candidates, totalling 24 fits


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV 1/3] END batch_size=8, epochs=10, model__activation=relu, model__optimizer=SGD;, score=0.979 total time=   0.5s


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV 2/3] END batch_size=8, epochs=10, model__activation=relu, model__optimizer=SGD;, score=0.965 total time=   0.5s


...

[CV 2/3] END batch_size=16, epochs=10, model__activation=tanh, model__optimizer=adam;, score=0.951 total time=   0.5s


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[CV 3/3] END batch_size=16, epochs=10, model__activation=tanh, model__optimizer=adam;, score=0.986 total time=   0.5s


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best Parameters: {'batch_size': 8, 'epochs': 10, 'model__activation': 'tanh', 'model__optimizer': 'adam'}
Best Score: 0.9765258215962441


In [22]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.976526 using {'batch_size': 8, 'epochs': 10, 'model__activation': 'tanh', 'model__optimizer': 'adam'}
0.969484 (0.006640) with: {'batch_size': 8, 'epochs': 10, 'model__activation': 'relu', 'model__optimizer': 'SGD'}
0.974178 (0.003320) with: {'batch_size': 8, 'epochs': 10, 'model__activation': 'relu', 'model__optimizer': 'adam'}
0.969484 (0.013279) with: {'batch_size': 8, 'epochs': 10, 'model__activation': 'tanh', 'model__optimizer': 'SGD'}
0.976526 (0.008783) with: {'batch_size': 8, 'epochs': 10, 'model__activation': 'tanh', 'model__optimizer': 'adam'}
0.974178 (0.003320) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'relu', 'model__optimizer': 'SGD'}
0.967136 (0.006640) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'relu', 'model__optimizer': 'adam'}
0.974178 (0.003320) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'tanh', 'model__optimizer': 'SGD'}
0.971831 (0.015213) with: {'batch_size': 16, 'epochs': 10, 'model__activation': 'tanh'